In [1]:
import pandas as pd

In [47]:
notes = pd.read_csv("comparison_simple_gpt3.5_2shot_v2.csv")

In [48]:
note_sample = notes.sample(1).squeeze()
print(note_sample)

Unnamed: 0                                                     1069
note_id                                               13472364-DS-7
hadm_id                                                    25686630
input              \nName:  ___                   Unit No:   ___...
output            Mr ___ is a ___ yr old male with metastatic ga...
careunit                                        Hematology/Oncology
eventtype                                                  transfer
curr_service                                                   OMED
gpt_completion    Mr. ___, a ___-year-old male with metastatic g...
Name: 69, dtype: object


In [49]:
print(note_sample['input'])

 
Name:  ___                   Unit No:   ___
 
Admission Date:  ___              Discharge Date:   ___
 
Date of Birth:  ___             Sex:   M
 
Service: MEDICINE
 
Allergies: 
clindamycin
 
Attending: ___.
 
Chief Complaint:
fever, abd pain
 
Major Surgical or Invasive Procedure:
ERCP ___
 
History of Present Illness:
___ with metastatic gastric cancer s/p neoadjuvant EOX, subtotal
gastrectomy ___, and progressed on FOLFOX, complicated by
malignant fistula to umbilicus with ostomy bag in place. 
He recently started ramucirumab paclitaxel on ___. Was last 
seen
by primary oncologist ___ at which time he reported a new
fistula to L lateral abdominal wall.

Pt's ___ called in today to report temp 101.6 temporal and HR
110-120s.  ostomy site reportedly looked clean. In ED it was
reported there was new blood in ostomy bag however pt denies 
this
and bag has not been changed today and is nonbloody. Also 
reports
ab pain esp in RLQ and surrounding his first fistula output 
site.
___ fist

In [50]:
print(note_sample['output'])

Mr ___ is a ___ yr old male with metastatic gastric Ca currently 
on weekly paclitaxel admitted with fever/SIRS likely colitis vs 
possible cholangitis. 

#Neutropenia/Fever/SIRS - Pt initially had fever and was started 
on cefepime/flagyl for intra-abdominal source, cholangitis vs 
colitis - colitis more likely as
EC output was more watery though this has improved, and pain not 
clearly in RUQ, also imaging not c/w cholangitis (though 
clinical dx) but rather simply biliary obstruction/dilation. CT 
without worsening fluid collection/abscess though this has been 
the issue in the past. See mgmt below for colitis/biliary 
obstruction.

# Colitis - progressive colonic wall thickening noted on CT ?___ 
ascites vs inflammatory/infectious process in setting of known 
malignant fistula. C.diff negative along with other stool 
studies. EC fistula output no longer watery by the time of 
discharge and no diarrhea. Neutropenia resolved and pt minimally 
symptomatic, cefepime/flagyl changed to p

In [51]:
print(note_sample['gpt_completion'])

Mr. ___, a ___-year-old male with metastatic gastric cancer and a history of multiple interventions for the disease including neoadjuvant EOX, subtotal gastrectomy, and FOLFOX therapy, presented to the emergency department with a chief complaint of fever and abdominal pain. He had a fever of 101.6°F and reported pain, primarily in the right lower quadrant and around his abdominal wall fistula sites. Initial evaluation revealed stable ascites and colonic wall thickening.

## ACTIVE ISSUES:
# Metastatic Gastric Cancer with Malignant Fistula
- Received cefepime and flagyl initially for fever and presumed infection.
- Imaging studies showed stable disease and no acute complications.
- Ostomy output noted to have increased yellow liquid with no signs of infection.
- Ongoing management with ramucirumab paclitaxel regimen.

# Abdominal Pain and Fistula Drainage
- Pain management with MS Contin and oxycodone.
- Persistent wound drainage management with gauze.
- Further investigations needed fo

# Calculate scores

In [42]:
import sys
import os

In [53]:
import os
import json
import numpy as np
import pandas as pd

import evaluate

from bleu import Bleu
from rouge import Rouge
from bertscore import BertScore  #


def calculate_scores_bhc(generated, reference, metrics):
    if not metrics:
        raise ValueError("No metrics specified for scoring.")
    print("Beginning scoring...")

    scores = {}
    for metric in metrics:
        scores[metric] = {"brief_hospital_course": []}

    # initialize scorers
    if "bleu" in metrics:
        bleuScorer = Bleu()
        print("bleuScorer initialized")
    if "rouge" in metrics:
        scores["rouge"]["brief_hospital_course"] = [[], [], []]
        rougeScorer = Rouge(["rouge1", "rouge2", "rougeL"])
        print("rougeScorer initialized")
    if "bertscore" in metrics:
        bertScorer = BertScore()
        print("bertScorer initialized")
    if "meteor" in metrics:
        meteorScorer = evaluate.load("meteor")
        print("meteorScorer initialized")

    def calculate_scores(rows_ref, rows_gen):
        if "bleu" in metrics:
            temp = bleuScorer(
                refs=rows_ref["brief_hospital_course"].tolist(),
                hyps=rows_gen["brief_hospital_course"].tolist(),
            )
            scores["bleu"]["brief_hospital_course"].extend(temp)
        if "rouge" in metrics:
            temp = rougeScorer(
                refs=rows_ref["brief_hospital_course"].tolist(),
                hyps=rows_gen["brief_hospital_course"].tolist(),
            )
            scores["rouge"]["brief_hospital_course"][0].extend(
                    temp['rouge1'],
            )
            scores["rouge"]["brief_hospital_course"][1].extend(
                    temp['rouge2'],
            )
            scores["rouge"]["brief_hospital_course"][2].extend(
                    temp['rougeL'],
            )
        if "bertscore" in metrics:
            temp = bertScorer(
                refs=rows_ref["brief_hospital_course"].tolist(),
                hyps=rows_gen["brief_hospital_course"].tolist(),
            )
            scores["bertscore"]["brief_hospital_course"].extend(temp)
        if "meteor" in metrics:
            temp = meteorScorer.compute(
                references=rows_ref["brief_hospital_course"].tolist(),
                predictions=rows_gen["brief_hospital_course"].tolist(),
            )
            scores["meteor"]["brief_hospital_course"].append(temp["meteor"])

        # print progress
        current_row = i + 128
        if current_row % 128 == 0:
            print(f"Processed {current_row}/{len(generated)} samples.", flush=True)

    reference.set_index("hadm_id", drop=False, inplace=True)
    generated.set_index("hadm_id", drop=False, inplace=True)

    batch_size = 128
    for i in range(0, len(generated), batch_size):
        rows_ref = reference[i : i + batch_size]
        rows_gen = generated[i : i + batch_size]
        calculate_scores(rows_ref=rows_ref, rows_gen=rows_gen)

    print(f"Processed {len(generated)}/{len(generated)} samples.", flush=True)
    print("Done.")
    return scores



ImportError: cannot import name 'Bleu' from 'bleu' (/home/vs428/.conda/envs/openai/lib/python3.12/site-packages/bleu/__init__.py)